<a href="https://colab.research.google.com/github/djolobolonjez/practice/blob/master/PSI_ML7_TF_IDF_(practice).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Initialization

In [ ]:
import os
import math
import sys
import nltk
#nltk.download('punkt')

from nltk.stem import SnowballStemmer 
from nltk.tokenize import word_tokenize, sent_tokenize

wordmap = {}
sentmap = {}
stemmer = SnowballStemmer("english")
docnum = 0

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


File opening for term frequency

In [ ]:
def file_open(fl):
  with open(fl, 'r') as f:
     for line in f:
       words = word_tokenize(line)
       for w in words:
         term = stemmer.stem(w.lower())
         if (term not in wordmap and term.isalnum()):
           wordmap[term] = 1
         else:
           if(term.isalnum()):
            wordmap[term] += 1

Parsing of the input

In [ ]:
directory, pth = input().split(" ")
path = os.path.split(pth)
title = path[-1]
os.chdir(path[0])
file_open(title)

/corpus,/corpus/skiing/History of skiing.txt


Document frequency

In [ ]:
def df_count(fl):
  with open(fl, 'r') as f:
    text = f.read()
    words = word_tokenize(text)
    
    for i in range(len(words)):
      words[i] = stemmer.stem(words[i].lower())
    for key in stems:
      if key in words:
        stems[key] += 1

Handling nested directories

In [ ]:
def dir_tree(dir):
  global docnum
  stack = [dir]

  while stack:
    pushed = 0
    dir = stack.pop()
    files = os.listdir(dir)
    os.chdir(dir)
    for file in files:
      if not file.endswith(".txt"):
        pushed += 1
        stack.append(file)
        
      else:
        docnum += 1
        df_count(file)
    if not pushed:
      path_parent = os.path.dirname(os.getcwd())
      os.chdir(path_parent)

Preparation for counting DF

In [ ]:
stems = wordmap.copy()
for key in stems:
  stems[key] = 0

for roots, dirs, files in os.walk(directory):
  for dir in dirs:
    os.chdir(directory)
    if not dir.startswith("."):
      dir_tree(dir)
  break

Conversion to IDF from DF

In [ ]:
def to_idf(stems):
  for key in stems:
    stems[key] = math.log((docnum/stems[key]), 10)

TF-IDF 

In [ ]:
def calculate_tfidf(wordmap, stems, result):
  wm = wordmap.copy()
  stm = stems.copy()
  for i in range(10):
    max = 0
    max_key = ""
    for key in stm:
      tf_idf = stm[key]*wm[key]
      if tf_idf > max:
        max = tf_idf
        max_key = key
    
    result.append([max_key, max])
    wm.pop(max_key, None)
    stm.pop(max_key, None)
    max = 0
    max_key = ""

Sorting some initial TF-IDF results

In [ ]:
result = []
to_idf(stems)
calculate_tfidf(wordmap, stems, result)

for i in range(len(result)-1):
  for j in range(i, len(result)):
    if(result[i][-1] == result[j][-1]):
      if(result[i][0] > result[j][0]):
        tmp = result[i][0]
        result[i][0] = result[j][0]
        result[j][0] = tmp

TF-IDF of sentences

In [ ]:
def sent_tfidf(words, sent):
  results = []
  wm = wordmap.copy()
  stm = stems.copy()
  added = []
  found = True
  for i in range(10):
    max = 0
    max_key = ""
    rmv = ""
    
    for word in words:

      wrd = stemmer.stem(word.lower())
      if wrd in added:
        max = stems[wrd]*wordmap[wrd]
        max_key = wrd
        rmv = word
        break
      else:
        tf_idf = stm[wrd]*wm[wrd]
        if tf_idf > max:
          max = tf_idf
          max_key = wrd
          rmv = word
    
    if rmv != "": 
      words.remove(rmv)
      added.append(max_key)
      results.append([max_key, max])
      wm.pop(max_key, None)
      stm.pop(max_key, None)
    max = 0
    max_key = ""
    rmv = ""

  stfidf = 0
  for res in results:
    stfidf += res[-1]
  sentmap[sent] = stfidf


Preparation for sentence tf-idf calculation

In [ ]:
def second_task(fl):
  
  sts = []
  with open(fl, 'r') as f:
    content = f.read()
    sentences = sent_tokenize(content)
    sts = list(sentences)
    for sent in sentences:
      words = word_tokenize(sent)
      words = [word for word in words if word.isalnum()]
      sent_tfidf(words, sent)
  
  return sts, dict(sorted(sentmap.items(), key = lambda item: item[1], reverse = True))

Sorting and printing out final data

In [ ]:
terms = []
for i in range(len(result)):
  terms.append(result[i][0])
os.chdir(path[0])
sts, secondmap = second_task(title)

sents = []
sentences = []
for key in secondmap.keys():
  sents.append(key)
  
for i in range(5):
  sentences.append(sents[i])
for i in range(len(sentences)-1):
  for j in range(i, len(sentences)):
    if(secondmap[sentences[i]] == secondmap[sentences[j]]):
      if(sts.index(sentences[i]) > sts.index(sentences[j])):
        tmp = sentences[i]
        sentences[i] = sentences[j]
        sentences[j] = tmp
print(*terms, sep = ", ")
print(*sentences, sep = " ")

ski, sami, norway, bce, norwegian, strap, skier, wax, club, bind
The Sami also have their own words for "skis" and "skiing": for example, the Lule Sami word for "ski" is sabek and skis are called sabega. As skiing became more specialized, so too did ski boots, leading to the splitting of designs between those for alpine skiing and cross-country skiing. Finnish has its own ancient words for skis and skiing: "ski" is suksi and "skiing" is hiihtää. For instance details of military ski exercises in the Danish-Norwegian army from 1767 are retained: Military races and exercises included downhill in rough terrain, target practice while skiing downhill, and 3 km cross-country skiing with full military backpack. The bottom of the short ski was either plain or covered with animal skin to aid this use, while the long ski supporting the weight of the skier was treated with animal fat in similar manner to modern ski waxing.
